<a href="https://colab.research.google.com/github/mehluli92/Sentiment_Analysis_Movie_Reviews/blob/main/Sentiment_Analysis_IMBD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [4]:
from google.colab import files
uploaded = files.upload()


Saving IMDB_Dataset.csv to IMDB_Dataset.csv


# New section

In [5]:
data = pd.read_csv("IMDB_Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.shape

(50000, 2)

In [7]:
type(data)

pandas.core.frame.DataFrame

In [8]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [9]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [10]:
# label encording positive = 1, negetive = 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data.head()

/tmp/ipython-input-10-252595779.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [11]:
# Use LSTM -> Long Short Term Memory
# RNN -> not good textual data because it has no memory
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [13]:
train_data.shape

(40000, 2)

In [14]:
test_data.shape

(10000, 2)

In [15]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [16]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [17]:
X_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [18]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [19]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [20]:
# LSTM Model Building

In [21]:
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2,recurrent_dropout = 0.2 ))
model.add(Dense(1, activation = "sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2 )

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 198s 383ms/step - accuracy: 0.7207 - loss: 0.5258 - val_accuracy: 0.8255 - val_loss: 0.3980
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 373ms/step - accuracy: 0.8483 - loss: 0.3599 - val_accuracy: 0.8594 - val_loss: 0.3353
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 390ms/step - accuracy: 0.8830 - loss: 0.2934 - val_accuracy: 0.8644 - val_loss: 0.3287
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 374ms/step - accuracy: 0.8980 - loss: 0.2622 - val_accuracy: 0.8724 - val_loss: 0.3209
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 207s 384ms/step - accuracy: 0.9136 - loss: 0.2243 - val_accuracy: 0.8675 - val_loss: 0.3575


In [25]:
model.save("model.h5")

In [28]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [26]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 111ms/step - accuracy: 0.8730 - loss: 0.3397


In [27]:
# Building Predictive System

In [31]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [35]:
predictive_system("This movies was amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


'positive'

In [36]:
predictive_system("A thrilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step


'positive'